# Scrape training data

In [1]:
import newspaper
from newspaper import Article
import pyPdf
import urllib2
import pandas as pd
import pickle

In [2]:
with open("un_traing.pkl", 'rb') as picklefile:
   urls = pickle.load(picklefile)

In [3]:
len(urls)

688

In [4]:
all_text = []

In [5]:
def download_file(url):
    try:
        response = urllib2.urlopen(url)
        file = open("document.pdf", 'wb')
        file.write(response.read())
        file.close()
    except:
        pass

In [6]:
def html_pages(url):
    info = {}
    try:
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()

        valid = article.is_valid_url()
        authors = article.authors
        date = article.publish_date
        text = article.text
        keywords = article.keywords
        summary = article.summary
        image = article.top_image

        info['url'] = url
        info['valid'] = valid
        info['authors'] = authors
        info['date'] = date
        info['text'] = text
        info['keywords'] = keywords
        info['summary'] = summary
        info['image'] = image

        all_text.append(info)
    except:
        pass

In [7]:
def getPDFContent(url):
    pdf_info = {}
    content = ""
    pdf = pyPdf.PdfFileReader(file("document.pdf", "rb"))
    for i in range(0, pdf.getNumPages()):
        content += pdf.getPage(i).extractText() + "\n"
        content = content.encode('ascii', 'ignore')
    pdf_info['url'] = url
    pdf_info['text'] = content
    all_text.append(pdf_info)

In [8]:
for url in urls:
    download_file(url)
    try:
        getPDFContent(url)
    except: 
        pass
    
    try:
        html_pages(url)
    except:
        continue

In [395]:
df = pd.DataFrame(all_text)

In [396]:
df.head()

,authors,date,image,keywords,summary,text,url,valid
0,NaN,NaN,NaN,NaN,NaN,United Nations S/2015/302 Security Council...,http://www.securitycouncilreport.org/atf/cf/%7...,NaN
1,[],None,,[],,,http://www.securitycouncilreport.org/atf/cf/%7...,False
2,[Heather Saul],2013-08-05 12:33:51+01:00,https://static.independent.co.uk/s3fs-public/t...,"[pakistan, homes, remote, afghanistan, floodin...",Flash flooding across Afghanistan and Pakistan...,Flash flooding across Afghanistan and Pakistan...,http://www.independent.co.uk/news/world/asia/1...,True
3,NaN,NaN,NaN,NaN,NaN,www.unocha.org The mission of the United Natio...,http://reliefweb.int/sites/reliefweb.int/files...,NaN
4,[],None,,[],,,http://reliefweb.int/sites/reliefweb.int/files...,False


In [397]:
df.shape

(880, 8)

In [401]:
import pickle

In [403]:
with open("df_training.pkl", 'wb') as picklefile:
    pickle.dump(df, picklefile)